In [ ]:
import spikeinterface.full as si

In [ ]:
%matplotlib widget

### Use simulated data for now

In [ ]:
rec, sort = si.generate_ground_truth_recording(durations=[60], num_channels=128, num_units=50)

In [ ]:
si.plot_probe_map(rec)

In [ ]:
si.plot_traces(rec, backend="ipywidgets")

### Extract waveforms

Start with simulated data

In [ ]:
we = si.extract_waveforms(rec, sort, folder=None, mode="memory", n_jobs=-1)

### Get spike scalings

In [ ]:
amp_scalings = si.compute_amplitude_scalings(we)

## Get residuals

In [ ]:
all_templates = we.get_all_templates()

In [ ]:
residuals_w_scaling = si.InjectTemplatesRecording(
    sorting=sort,
    templates=-all_templates,
    amplitude_factor=amp_scalings,
    parent_recording=rec
)
residuals_wout_scaling = si.InjectTemplatesRecording(
    sorting=sort,
    templates=-all_templates,
    amplitude_factor=None,
    parent_recording=rec
)

In [ ]:
rec_dict = dict(
    original=rec,
    residuals_w=residuals_w_scaling,
    residuals_wout=residuals_wout_scaling
)

In [ ]:
si.plot_traces(rec_dict, backend="ipywidgets")

# Use a local NWB file

In [ ]:
recording = si.read_nwb_recording(
    "/home/alessio/Documents/Data/IBL/sub-CSHL045_ses-46794e05-3f6a-4d35-afb3-9165091a5a74_behavior+ecephys+image.nwb",
    electrical_series_path="acquisition/ElectricalSeriesAp"
)

In [ ]:
recording

In [ ]:
sorting = si.read_nwb_sorting(
    "/home/alessio/Documents/Data/IBL/sub-CSHL045_ses-46794e05-3f6a-4d35-afb3-9165091a5a74_behavior+ecephys+image.nwb",
    electrical_series_path="acquisition/ElectricalSeriesAp"
)

In [ ]:
sorting

In [ ]:
clip_minutes = 5

In [ ]:
sampling_frequency = recording.sampling_frequency

In [ ]:
start_time = 1200
start_frame = int(start_time * sampling_frequency)
end_frame = int((start_time + clip_minutes * 60)* sampling_frequency)

In [ ]:
recording_c = recording.frame_slice(start_frame, end_frame)
sorting_c = sorting.frame_slice(start_frame, end_frame)

In [ ]:
recording_processed = si.highpass_filter(recording_c)
recording_processed = si.common_reference(recording_processed)

In [ ]:
we = si.extract_waveforms(recording_processed, sorting_c, mode="memory", n_jobs=-1)

In [ ]:
len(we.unit_ids)

In [ ]:
si.plot_rasters(sorting_c)

In [ ]:
amp_scalings = si.compute_amplitude_scalings(we)

In [ ]:
sorting_c.get_property_keys()

In [ ]:
recording_scaled = si.scale(recording_processed, gain=recording.get_channel_gains(), dtype="float32")

In [ ]:
all_templates = we.get_all_templates()

In [ ]:
residuals_w_scaling = si.InjectTemplatesRecording(
    sorting=sorting_c,
    templates=-all_templates,
    amplitude_factor=amp_scalings,
    nbefore=we.nbefore,
    parent_recording=we.recording
)
residuals_wout_scaling = si.InjectTemplatesRecording(
    sorting=sorting_c,
    templates=-all_templates,
    amplitude_factor=None,
    nbefore=we.nbefore,
    parent_recording=we.recording
)

In [ ]:
rec_dict = dict(
    original=recording_processed,
    residuals_w=residuals_w_scaling,
    residuals_wout=residuals_wout_scaling
)

In [ ]:
all_templates.shape

In [ ]:
si.plot_rasters(sorting_c)

In [ ]:
spike_counts = sorting_c.count_num_spikes_per_unit()

In [ ]:
spike_counts

In [ ]:
recording_c.get_times()[0]

In [ ]:
len(sorting_c.get_unit_spike_train("115", return_times=True))

In [ ]:
for unit in we.unit_ids:
    

In [ ]:
times_155 = we.sorting.get_unit_spike_train("115", return_times=True)

In [ ]:
len(times_155)

In [ ]:
print(times_155[0], times_155[-1])

In [ ]:
si.plot_unit_templates(we, backend="ipywidgets")

In [ ]:
si.plot_traces(rec_dict, backend="ipywidgets")

In [ ]:
for unit in sorting_c.unit_ids:
    print(unit, len(sorting_c.get_unit_spike_train(unit)))

## Detection analysis

In [ ]:
from spikeinterface.core.node_pipeline import 

# Stream NWB

### Load IBL recording

In [ ]:
rec_ibl = si.read_nwb_recording(
    file_path="https://api.dandiarchive.org/api/assets/b2fee6b4-6049-4720-8d3d-44f8ec483527/download/",
    stream_mode="remfile",
    electrical_series_path="acquisition/ElectricalSeriesAp00"
)

In [ ]:
rec_f = si.highpass_filter(rec_ibl)
rec_de = si.highpass_spatial_filter(rec_f)
rec_cmr = si.common_reference(rec_f)

In [ ]:
rec_dict = dict(
    raw=rec_ibl,
    hp=rec_f,
    destripe=rec_de,
    cmr=rec_cmr
)

In [ ]:
si.plot_traces(rec_dict, backend="ipywidgets")

### Save 10 s data

In [ ]:
rec_ibl_cut = rec_ibl.frame_slice(0, int(10 * rec_ibl.sampling_frequency))

In [ ]:
rec_ibl_saved = rec_ib_cut.save(folder="ibl_snippet_10", n_jobs=-1)

In [ ]:
rec_f = si.highpass_filter(rec_ibl_cut)
rec_de = si.highpass_spatial_filter(rec_f)
rec_cmr = si.common_reference(rec_f)

In [ ]:
debug

In [ ]:
recording.sampling_frequency

### Load IBL recording

In [ ]:
rec_ibl = si.read_nwb_recording(
    file_path="https://api.dandiarchive.org/api/assets/b2fee6b4-6049-4720-8d3d-44f8ec483527/download/",
    stream_mode="remfile",
    electrical_series_path="acquisition/ElectricalSeriesAp00"
)

In [ ]:
rec_f = si.highpass_filter(rec_ibl)
rec_de = si.highpass_spatial_filter(rec_f)
rec_cmr = si.common_reference(rec_f)

In [ ]:
rec_dict = dict(
    raw=rec_ibl,
    hp=rec_f,
    destripe=rec_de,
    cmr=rec_cmr
)

In [ ]:
si.plot_traces(rec_dict, backend="ipywidgets")

### Save 10 s data

In [ ]:
rec_ibl_cut = rec_ibl.frame_slice(0, int(10 * rec_ibl.sampling_frequency))

In [ ]:
rec_ibl_saved = rec_ib_cut.save(folder="ibl_snippet_10", n_jobs=-1)

In [ ]:
rec_f = si.highpass_filter(rec_ibl_cut)
rec_de = si.highpass_spatial_filter(rec_f)
rec_cmr = si.common_reference(rec_f)

## Load IBL sorting

In [ ]:
from one.api import ONE
from brainbox.io.one import SpikeSortingLoader
one = ONE(base_url='https://openalyx.internationalbrainlab.org')

In [ ]:
pid = '410ecab4-1d7b-4c57-85a8-dfebf29feba8'
sl = SpikeSortingLoader(pid=pid, one=one)
spikes, clusters, channels = sl.load_spike_sorting()

In [ ]:
sort_ibl_cut = sort_ibl.frame_slice(0, int(10 * rec_ibl.sampling_frequency))
sort_ibl_saved = sort_ibl_cut.save(folder="ibl_sort_snippet_10")

In [ ]:
sort_ibl_cut = sort_ibl.frame_slice(0, int(10 * rec_ibl.sampling_frequency))
sort_ibl_saved = sort_ibl_cut.save(folder="ibl_sort_snippet_10")

In [ ]:
pid = '410ecab4-1d7b-4c57-85a8-dfebf29feba8'
sl = SpikeSortingLoader(pid=pid, one=one)
spikes, clusters, channels = sl.load_spike_sorting()

In [ ]:
from one.api import ONE
from brainbox.io.one import SpikeSortingLoader
one = ONE(base_url='https://openalyx.internationalbrainlab.org')

## Load IBL sorting

In [ ]:
rec_f = si.highpass_filter(rec_ibl_cut)
rec_de = si.highpass_spatial_filter(rec_f)
rec_cmr = si.common_reference(rec_f)